In [1]:
import pandas as pd
import numpy as np

In [2]:
import MySQLdb
mysql_cn= MySQLdb.connect(host='127.0.0.1', 
                port=3306,user='root', passwd='0715sql', 
                db='tianchi')

In [3]:
furture=pd.read_sql('select * from test_set;', con=mysql_cn)
past= pd.read_sql('select * from train_set;', con=mysql_cn)

In [4]:
mysql_cn.close()

In [21]:
past_buy=past.loc[past["buy"]==1,]
#past_buy=pd.read_sql('select * from train_set where buy = 1;', con=mysql_cn)

In [22]:
past_view=past.loc[past["buy"]==0,]
#past_view=pd.read_sql('select * from train_set where buy = 0;', con=mysql_cn)

In [23]:
#test set
x,y=53,10000
sample1=np.random.choice(len(past_buy),x,replace=False)
test=past_buy.iloc[sample1,:]
past_buy=past_buy.drop(past_buy.index[sample1])
del sample1

sample2=np.random.choice(len(past_view),y,replace=False)
test=test.append(past_view.iloc[sample2,:])
past_view=past_view.drop(past_view.index[sample2])

del sample2

In [24]:
from sklearn import tree

In [11]:
n,m=800,8

In [26]:
cluster=()
for i in range(0,n):
    cluster+=(tree.DecisionTreeClassifier(),)

In [25]:
features=()
for i in range(0,n):
    select=np.random.choice(range(4,206),m,replace=False)
    features+=(np.append(select,3),)
del select

In [27]:
k=10
a=103#-x
b=k*a

In [28]:
trains=()
for i in range(0,n):
    sampleA=np.random.choice(len(past_buy),a,replace=True)
    train=past_buy.iloc[sampleA,features[i]]
    
    sampleB=np.random.choice(len(past_view),b,replace=True)
    train=train.append(past_view.iloc[sampleB,features[i]])
    
    trains=trains+(train,)
del sampleA
del sampleB

In [29]:
fitted=()
for i in range(0,n):
    fitted+=(cluster[i].fit(trains[i].iloc[:,range(m)],trains[i]["buy"]),)

In [30]:
evalu=()
for i in range(0,n):
    select=features[i][range(m)]
    evalu=evalu+(fitted[i].predict(test.iloc[:,select]),)
del select

In [32]:
check=evalu[0]

for i in range(1,n):
    check=check+evalu[i]

In [79]:
truth,dream=np.array(test["buy"]),(check>(float(n)/2)).astype(int)

In [80]:
sum(dream[truth==1])

16

In [81]:
sum(dream[truth==0])

5

In [92]:
predict=()
for i in range(0,n):
    select=features[i][range(m)]-1
    predict=predict+(fitted[i].predict(furture.iloc[:,select]),)

In [93]:
final=predict[0]
for i in range(1,n):
    final=final+predict[i]

In [94]:
res=furture.loc[final>=n*1/2,["user_id","item_id"]]
len(res)

22

In [95]:
res.to_csv("C:/data science/race and projects/tianchi/tianchi_mobile_recommendation_predict.csv",index=False,encoding="UTF-8")